In [1]:
import re, os, glob, csv, math, requests, time, sys, random, json, datetime, urllib
import pandas as pd
from bs4 import BeautifulSoup as bs
from selenium import webdriver
from sklearn.utils import shuffle      
import numpy as np
from nordvpn_switcher import initialize_VPN,rotate_VPN,terminate_VPN
from webdriver_manager.chrome import ChromeDriverManager

from webscraping_kit import write_htmlfile, write_json_tofile, write_driverhtmlfile
from webscraping_kit import read_jsoncsv, read_htmlfile, read_htmlresponse, read_driverresponse
from webscraping_kit import rmnl, get_apikeys, get_cases
from htmlparsing_kit import get_oldcaseids, get_collected

mainpath = str(os.getcwd())+'/'
casefolder = mainpath + 'data/cases/'
headers = get_apikeys(mainpath + 'apikeys.txt','headers')

df = pd.read_csv(mainpath + 'output/1_caseidx.csv',sep='\t')
print(df.shape)
print(df['source'].value_counts())
# tablecaseidx.csv should be 36,437, -1 for the case from 2005, and -2 for the cases missing from the canlii tables = 36434

(36499, 5)
both         33139
webscrape     3295
api             65
Name: source, dtype: int64


<h2> handling collected cases </h2>

In [2]:
""" get cases already collected
- using a list of folders sorted by oldest collection first, keep only the cases from the newest collection if it appears in others
"""
oldcasefolder = mainpath + 'data/oldcases/'
oldcasefolders = [oldcasefolder+ "cases_main/*.html",oldcasefolder+ "cases/*.html",oldcasefolder+ "cases_add/*.html"]

odf = get_collected(oldcasefolders)

df['collected'] = df['cid'].map(dict(zip(odf['cid'],odf['oldcasefiles'])))
df['collected'].fillna('missing',inplace=True)
print(df[df['collected']=='missing'].shape)

df['goodcasefiles'] = np.where(df['collected']=='missing',df['caseoutfile'],df['collected'])
print(len(list(set(df['goodcasefiles'].tolist()))))

# df.to_csv(mainpath + 'output/2_casefileidx.csv',sep='\t',index=False,quoting=csv.QUOTE_ALL)

(131, 6)
36499


In [3]:
df_tocollect = df[df['collected']=='missing']
print(df_tocollect.shape)
print(df.shape)

(131, 7)
(36499, 7)


<h2>webscraping cases using the urls collected from the api and website</h2>

In [4]:
caselinkdict = dict(zip(df_tocollect['caseoutfile'],df_tocollect['caseurl']))
print(len(caselinkdict))

settings = initialize_VPN(save=0,area_input=['complete rotation'],skip_settings=1)

rotate_VPN(settings,google_check=1)
# terminate_VPN(settings)

131

You're using Linux.
Performing system check...
###########################

NordVPN installation check: ✓
NordVPN login check: ✓

You've entered a list of connection options. Checking list...


Fetching list of all current NordVPN servers...


Done!


Your current ip-address is: 142.59.160.19

Connecting you to jp523 ...

Found a server! You're now on Japan #523
your new ip-address is: 212.102.50.207

Performing captcha-check on Google search and Youtube...
---------------------------
Google and YouTube don't throw any Captcha's: ✓

Done! Enjoy your new server.



In [5]:
get_cases(caselinkdict,settings)



====== WebDriver manager ======
Current google-chrome version is 96.0.4664
Get LATEST chromedriver version for 96.0.4664 google-chrome
Driver [/home/bunds/.wdm/drivers/chromedriver/linux64/96.0.4664.45/chromedriver] found in cache


====== WebDriver manager ======
Current google-chrome version is 96.0.4664
Get LATEST chromedriver version for 96.0.4664 google-chrome
Driver [/home/bunds/.wdm/drivers/chromedriver/linux64/96.0.4664.45/chromedriver] found in cache


====== WebDriver manager ======
Current google-chrome version is 96.0.4664
Get LATEST chromedriver version for 96.0.4664 google-chrome
Driver [/home/bunds/.wdm/drivers/chromedriver/linux64/96.0.4664.45/chromedriver] found in cache


====== WebDriver manager ======
Current google-chrome version is 96.0.4664
Get LATEST chromedriver version for 96.0.4664 google-chrome
Driver [/home/bunds/.wdm/drivers/chromedriver/linux64/96.0.4664.45/chromedriver] found in cache


====== WebDriver manager ======
Current google-chrome version is 9

In [6]:
terminate_VPN(settings)


Disconnecting...
Done!
